# libraries 

In [ ]:
import requests #The requests library for HTTP requests in Python
import csv
import pandas as pd
from secrets import IEX_CLOUD_API_TOKEN_TEST
from secrets import IEX_CLOUD_API_TOKEN_LIVE


# Get a url of a stock with extended hours data (MINE)

In [ ]:
symbol = 'AAPL'
# api_url = f'https://sandbox.iexapis.com/stable/stock/goog/chart/date/20201208?chartByDay=true&token={IEX_CLOUD_API_TOKEN}'
api_url = f'https://sandbox.iexapis.com/stable/stock/TWTR/quote?token={IEX_CLOUD_API_TOKEN_TEST}'
data = requests.get(api_url).json()
data

# From their website

https://iexcloud.zendesk.com/hc/en-us/articles/1500011249702-Historical-Stock-Prices-on-IEX-Cloud

In [ ]:
api_url = 'https://cloud.iexapis.com/stable/stock/xom/chart/date/20200415?token=sk_733fc5bb33d8422da50a3018daa5db74'
data = requests.get(api_url).json()
print(data)



# The download_IEX.py API end point

In [ ]:
api_url = 'https://cloud.iexapis.com/stable/stock/twtr/chart/date/20200415/?&token=sk_733fc5bb33d8422da50a3018daa5db74'
data = requests.get(api_url).json()
print(data)



# Alpha API

In [ ]:
ticker = 'BBIG'
date= 'year1month1'
apiKey = '5MACTR59FOK093UR'

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
CSV_URL = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol={ticker}&outputsize=full&interval=1min&slice=year1month1&apikey={apiKey}'

with requests.Session() as s:
    download = s.get(CSV_URL)
    decoded_content = download.content.decode('utf-8')
    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    my_list = list(cr)
    for row in my_list:
        print(row)



# Get a list of tickers in the NASDAQ (IEX cloud)

In [25]:
api_url = f'https://cloud.iexapis.com/stable/ref-data/iex/symbols?format=csv&token={IEX_CLOUD_API_TOKEN_LIVE}'
with requests.Session() as s:
    download = s.get(api_url)

    decoded_content = download.content.decode('utf-8')

    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    my_list = list(cr)
    for row in my_list:
        print(row)


['symbol', 'date', 'isEnabled', 'name']
['A', '2022-01-18', 'true', 'AGILENT TECHNOLOGIES INC']
['AA', '2022-01-18', 'true', 'ALCOA CORP']
['AAA', '2022-01-18', 'true', 'AAF FIRST PRIORITY CLO BOND']
['AAAU', '2022-01-18', 'true', 'GOLDMAN SACHS PHYSICAL GOLD']
['AAC', '2022-01-18', 'true', 'ARES ACQUISITION CORP-A']
['AAC+', '2022-01-18', 'true', '']
['AAC=', '2022-01-18', 'true', 'ARES ACQUISITION CORP']
['AACG', '2022-01-18', 'true', 'ATA CREATIVITY GLOBAL - ADR']
['AACI', '2022-01-18', 'true', 'ARMADA ACQUISITION CORP I']
['AACIU', '2022-01-18', 'true', 'ARMADA ACQUISITION CORP I']
['AACIW', '2022-01-18', 'true', '']
['AADI', '2022-01-18', 'true', 'AADI BIOSCIENCE INC']
['AADR', '2022-01-18', 'true', 'ADVISORSHARES DORSEY WRIGHT']
['AAIC', '2022-01-18', 'true', 'ARLINGTON ASSET INVESTMENT-A']
['AAIC-B', '2022-01-18', 'true', '']
['AAIC-C', '2022-01-18', 'true', '']
['AAIN', '2022-01-18', 'true', '']
['AAL', '2022-01-18', 'true', 'AMERICAN AIRLINES GROUP INC']
['AAM-A', '2022-01-18'